In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
data=pd.read_csv('C:/Users/astle/Downloads/gps20.cat',sep='\n',skiprows=4)

In [2]:
data

,Long Lat RA Dec Sprob Fpeak Fint RMS Maj Min PA Field OldGPS
0,339.739+0.419 16 44 12.282 -45 10 17.79 0.17 ...
1,339.968+0.555 16 44 28.207 -44 54 36.14 0.06 ...
2,339.980-0.539 16 49 14.776 -45 36 32.25 0.02 ...
3,340.025-0.002 16 47 04.495 -45 13 41.51 0.60 ...
4,340.051-0.231 16 48 09.875 -45 21 25.84 0.02 ...
...,...
5040,119.012+0.029 00 17 13.202 +62 38 25.25 0.02 ...
5041,119.071+0.027 00 17 44.083 +62 38 45.97 0.02 ...
5042,119.339-0.504 00 20 35.979 +62 09 08.99 0.02 ...
5043,119.450+0.401 00 20 37.076 +63 03 50.31 0.23 ...


In [3]:
data.iloc[:,0][913][:13].split('-')[0]
data.iloc[:,0][913][81:86]

' 1.03'

In [20]:
query,size=[],[]
for i in range (len(data)):
    l=float(data.iloc[:,0][i][:7])
    b=float(data.iloc[:,0][i][8:13])
    c=float(data.iloc[:,0][i][81:86])
    if l<48.5 and l>5:
        if abs(b)<0.8:
            query.append([l,data.iloc[:,0][i][7:13]])
            size.append(c)

In [21]:
size

[2.18,
 1.06,
 0.0,
 1.49,
 0.71,
 0.0,
 2.0,
 6.18,
 2.85,
 41.4,
 18.14,
 0.0,
 0.0,
 1.67,
 4.88,
 1.74,
 2.38,
 6.03,
 2.71,
 1.83,
 7.09,
 9.76,
 10.02,
 32.63,
 47.96,
 11.88,
 18.85,
 10.2,
 15.29,
 1.53,
 0.9,
 11.33,
 1.53,
 0.88,
 2.29,
 3.06,
 0.0,
 1.84,
 0.0,
 2.62,
 4.28,
 20.62,
 1.23,
 0.84,
 0.0,
 1.79,
 9.39,
 7.06,
 9.61,
 5.33,
 2.16,
 1.08,
 0.0,
 2.67,
 5.83,
 17.16,
 3.46,
 35.54,
 23.25,
 12.5,
 9.61,
 1.25,
 2.37,
 0.0,
 3.1,
 5.09,
 0.0,
 3.02,
 4.68,
 17.87,
 14.68,
 18.92,
 11.56,
 4.67,
 2.89,
 0.0,
 1.89,
 1.54,
 2.01,
 0.0,
 0.0,
 1.68,
 0.67,
 0.0,
 0.0,
 5.19,
 1.81,
 2.95,
 0.0,
 0.0,
 0.0,
 0.0,
 1.26,
 0.0,
 1.53,
 6.45,
 12.96,
 2.35,
 1.51,
 9.72,
 3.68,
 7.29,
 3.93,
 15.49,
 4.75,
 6.09,
 5.03,
 10.16,
 5.96,
 4.16,
 4.65,
 7.13,
 5.29,
 6.02,
 7.07,
 8.96,
 7.61,
 1.61,
 6.1,
 15.92,
 4.24,
 5.44,
 10.97,
 3.56,
 0.0,
 8.97,
 5.24,
 11.17,
 3.96,
 9.56,
 5.64,
 4.85,
 1.88,
 1.04,
 4.36,
 0.0,
 3.55,
 16.89,
 2.0,
 4.79,
 2.86,
 9.26,
 1.62,
 5.

In [22]:
indices=[]
for i,x in enumerate(size):
    if x<5 or x>30:
        indices.append(i)
indices=sorted(indices,reverse=True)#reverse sort so that the indices of the elements to remove from query isnt affected

In [23]:
for i in indices:
    del query[i]
    del size[i]

In [70]:
testq=query[60:80]
tests=size[60:80]

In [71]:
for i in testq:
    i[1]=float(i[1])

In [72]:
driver=webdriver.Firefox()
driver.get('https://third.ucllnl.org/cgi-bin/gpscutout')
script='arguments[0].value=arguments[1]'

In [73]:
j=0
for i in range(len(testq)):
    try:
        RA=driver.find_element_by_name('RA')
        Size=driver.find_element_by_name('ImageSize')
        driver.execute_script(script, RA,'{} {}'.format(testq[i][0],testq[i][1]))
        driver.execute_script(script,Size,'{}'.format(tests[i]))
        
        exg=driver.find_element_by_name('Equinox')
        driver.execute_script(script,exg,'{}'.format('Galactic'))
        
        download=driver.find_elements_by_name('ImageType')[2]
        download.click()
        time.sleep(2)
        submit=driver.find_element_by_name('.submit')
        submit.click()
    except NoSuchElementException:
        print('missing file with Galactic coordinates: {}'.format(testq[i]))
        driver.get('https://third.ucllnl.org/cgi-bin/gpscutout')
        time.sleep(1)
        j+=1
        continue
print('missing files percentage: '+ str((round(100*j*len(testq)**-1,2))) +'%')

missing files percentage: 0.0%
